# Iteration mechanisms

In [1]:
import numpy as np
from collections.abc import Iterable, Iterator

## Pattern: Iterable et Iterator
Comme beaucoup de langages, Python définit le pattern Iterable / Iterator mais à sa façon.

### exemples de données builtin iterables

In [2]:
villes = ['Lyon', 'Toulouse', 'Balma', 'Pau']
ville = {
    'nom': 'Toulouse',
    'population': 500_000,
    'code_postal': '31000'
}
dicton = 'Le blé semé le jour de Saint-Bruno devient noir.'

In [3]:
villes.__iter__
ville.__iter__
dicton.__iter__

<method-wrapper '__iter__' of str object at 0x0000024C4C129300>

### Iterator détaillé en pas à pas

In [4]:
it = iter(villes)
it

In [5]:
it.__next__

<method-wrapper '__next__' of list_iterator object at 0x0000024C4C34FA90>

In [6]:
# execute several times this cell
# iteration ends with exception: StopIteration
next(it)

'Lyon'

### Itération par boucle for

In [7]:
for v in villes:
    print(v)

Lyon
Toulouse
Balma
Pau


In [8]:
# => TypeError: 'int' object is not iterable
# nb = 123
# for digit in nb:
#     print(digit)

In [9]:
# https://docs.python.org/3/library/collections.abc.html
# définit les classes abstraites: Iterable, Sized, Sequence, Collection

assert isinstance(villes, list)
assert isinstance(villes, Iterable)

### Exemple du tableau numpy

In [10]:
data = np.random.normal(10.0, 2.5, 10_000)
data

array([11.59517062,  6.31469485,  5.34240645, ..., 10.10449915,
        9.58516164, 10.31155724], shape=(10000,))

In [11]:
type(data)

numpy.ndarray

In [12]:
assert isinstance(data, Iterable)

### Objet Dummy iterable à l'ancienne
Pour être plus propre, il faudrait hériter
de la class abstrait Iterable, mais concrètement les mécanismes d'itérations de python ne demande que le hook `__iter__`

In [13]:
class DummyIterable:

    def __iter__(self):
        return iter([])

In [14]:
d = DummyIterable()
for v in d:
    print('never')

## Specifité python du pattern Iterator-Iterable
En Python, tout iterateur est iterable (idempotence du pattern)
en se renvoyant lui même.

Cela permet à plein de fonction python de se contenter de déclarer
accepter un iterable (docstring, type hint) et d'accepter en paramètre réel aussi
bien un iterable qu'un iterator.

In [15]:
it = iter(dicton)
it2 = iter(it)
assert it is it2 # idempotence : l'iterateur d'un iterateur est lui même
assert isinstance(it, Iterator)
assert isinstance(it, Iterable)

print('Début 1er parcours')
for l in it:
    print(l)
print('Fin 1er parcours')
print()
print('Début 2e parcours')
for l in it2: # empty iteration (it2 est déjà consommé via it)
    print(l)
print('Fin 2e parcours')

Début 1er parcours
L
e
 
b
l
é
 
s
e
m
é
 
l
e
 
j
o
u
r
 
d
e
 
S
a
i
n
t
-
B
r
u
n
o
 
d
e
v
i
e
n
t
 
n
o
i
r
.
Fin 1er parcours

Début 2e parcours
Fin 2e parcours


## Iteration via une fonction builtin
Beaucoup de fonctions builtins acceptent des objets `iterable` en paramètre: list, sum, min, max, zip, ...

In [16]:
sum(data)

np.float64(100024.88267997405)

In [17]:
sum?

Signature: sum(iterable, /, start=0)
Docstring:
Return the sum of a 'start' value (default: 0) plus an iterable of numbers

When the iterable is empty, return the start value.
This function is intended specifically for use with numeric values and may
reject non-numeric types.
Type:      builtin_function_or_method

In [18]:
zip?

Init signature: zip(*iterables, strict=False)
Docstring:     
The zip object yields n-length tuples, where n is the number of iterables
passed as positional arguments to zip().  The i-th element in every tuple
comes from the i-th iterable argument to zip().  This continues until the
shortest argument is exhausted.

If strict is true and one of the arguments is exhausted before the others,
raise a ValueError.

   >>> list(zip('abcdefg', range(3), range(4)))
   [('a', 0, 0), ('b', 1, 1), ('c', 2, 2)]
Type:           type
Subclasses:     

In [19]:
for v, ld, lv in zip(villes, dicton, ville):
    print(v, ld, lv, sep='#')

Lyon#L#nom
Toulouse#e#population
Balma# #code_postal


In [20]:
# NB: read functions of pandas have some iterable arguments
# pd.read_csv?

## Generators
yield values one by one

In [21]:
g = (v.upper() for v in villes)
g

<generator object <genexpr> at 0x0000024C4C4564D0>

In [22]:
next(g)

'LYON'

In [23]:
assert isinstance(g, Iterable)
assert isinstance(g, Iterator)

In [24]:
# sum([len(v) for v in villes])
sum(len(v) for v in villes)

20

In [25]:
def gen_v():
    yield 0
    yield 1
    yield 2

In [26]:
g = gen_v()
g

<generator object gen_v at 0x0000024C4C451C70>

In [27]:
next(g)

0

In [28]:
for v in gen_v():
    print(v)

0
1
2


In [29]:
values = list(gen_v())
values

[0, 1, 2]

In [30]:
sum(gen_v())

3

### Atelier: Générateur suite de Fibonacci
- V1: jusqu'au terme n
  ```
  termes = list(fibo(10))
  
  ``` 
- V2: infini
  ```
  g = fibo2()
  ...
  ```

In [31]:
# V2: generateur infini
def fibonacci():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b
        
# Afficher les 10 premiers nombres de Fibonacci
fib_gen = fibonacci()
for _ in range(10):
    print(next(fib_gen))

0
1
1
2
3
5
8
13
21
34


In [32]:
# Exploitation du générateur infini jusqu'au crash:

# fib_gen = fibonacci()
# while True:
#     print(next(fib_gen), end=' ,')

In [33]:
# V1: générateur fini
def fibonacci_n(n):
    a, b = 0, 1
    for _ in range(n):
        yield a
        a, b = b, a + b

In [34]:
print(list(fibonacci_n(0)))
print(list(fibonacci_n(1)))
print(list(fibonacci_n(2)))
print(list(fibonacci_n(3)))
print(list(fibonacci_n(10)))

[]
[0]
[0, 1]
[0, 1, 1]
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


In [35]:
g = fibonacci_n(0)
assert iter(g) is g